In [4]:
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done


In [16]:
#Import spark packages
from pyspark import SparkFiles
from pyspark.sql import SparkSession

In [15]:
#import pandas
import pandas as pd

In [9]:
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.17.jar").getOrCreate()

In [10]:
#read tables from files
TBurl = "https://raw.githubusercontent.com/sbogue2/Final_Project/main/title_basics.csv"
spark.sparkContext.addFile(TBurl)

In [12]:
title_basics_df = spark.read.csv(SparkFiles.get("title_basics.csv"), sep = "\t", header = True, inferSchema=True)

In [14]:
title_basics_df.show()

+---------+---------+--------------+--------------------+
|   tconst|startYear|runtimeMinutes|              genres|
+---------+---------+--------------+--------------------+
|tt0000009|     1894|            45|             Romance|
|tt0000147|     1897|           100|Documentary,News,...|
|tt0000502|     1905|           100|                null|
|tt0000574|     1906|            70|Action,Adventure,...|
|tt0000591|     1907|            90|               Drama|
|tt0000679|     1908|           120|   Adventure,Fantasy|
|tt0000941|     1909|            45|               Drama|
|tt0001184|     1910|            58|     Adventure,Drama|
|tt0001285|     1909|            50|Biography,Drama,F...|
|tt0001498|     1911|            51|                 War|
|tt0001592|     1911|            52|               Drama|
|tt0001614|     1911|            60|               Drama|
|tt0001630|     1911|            45|                null|
|tt0001756|     1911|            92|                null|
|tt0001790|   

In [26]:
# take a 5% sample of the data for our test run
movies_sample = title_basics_df.sample(fraction = 0.05, seed = 1735)

In [27]:
# the amount of rows in the sample
movies_sample.count()

19651

In [28]:
movies_sample_pd = movies_sample.toPandas()

In [30]:
movies_sample_pd

,tconst,startYear,runtimeMinutes,genres
0,tt0000009,1894.0,45,Romance
1,tt0001756,1911.0,92,None
2,tt0002445,1913.0,120,"Drama,History"
3,tt0002917,1913.0,51,Drama
4,tt0003227,1913.0,54,Drama
...,...,...,...,...
19646,tt9910040,1981.0,70,None
19647,tt9913660,2019.0,102,Drama
19648,tt9916124,1997.0,108,Documentary
19649,tt9916190,2020.0,95,"Action,Adventure,Thriller"


In [31]:
movies_sample_clean = movies_sample_pd.drop('tconst', axis = 1)

,startYear,runtimeMinutes,genres
0,1894.0,45,Romance
1,1911.0,92,None
2,1913.0,120,"Drama,History"
3,1913.0,51,Drama
4,1913.0,54,Drama
...,...,...,...
19646,1981.0,70,None
19647,2019.0,102,Drama
19648,1997.0,108,Documentary
19649,2020.0,95,"Action,Adventure,Thriller"
